In [ ]:
!pip install torchxrayvision
!pip install numpy
!pip install pillow
!pip install pydicom

import torchxrayvision

In [ ]:
from re import L
from pyparsing.helpers import PositionToken
from pandas import Index
#Code is first test on https://github.com/mlmed/torchxrayvision
import torchxrayvision as xrv
import skimage, torch, torchvision
import matplotlib.pyplot as plt
import os
import tabulate
import numpy as np
import pydicom
import cv2 as cv

#added code
from google.colab import drive
from PIL import Image
from os import listdir
from tabulate import tabulate
from PIL import Image
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from datetime import datetime, date
drive.mount('/content/drive')
#end of added code

def calculate_age(birth_date, current_date):
    # Calculation
    years = current_date.year - birth_date.year
    months = current_date.month - birth_date.month
    days = current_date.day - birth_date.day

    # Adjust for negative differences
    if days < 0:
        months -= 1
        days += get_days_in_month(birth_date.month, birth_date.year)
    if months < 0:
        years -= 1
        months += 12

    return years, months, days

def get_days_in_month(month, year):
    # Returns the number of days in a given month and year
    if month == 2:  # February
        if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0):
            return 29  # Leap year
        else:
            return 28
    elif month in [4, 6, 9, 11]:  # April, June, September, November
        return 30
    else:
        return 31

def dcm_patient_data(path_file):
  dcm_data = pydicom.dcmread(path_file)
  patient_data = {}
  have_patient_dob = False
  have_acquisition_date = False
  patient_birth_date = ""
  patient_age_now = ""
  acquisition_date = ""
  acquisition_age = ""

  patient_data.update({"Study ID" :dcm_data.StudyID})
  patient_data.update({"Patient ID" :dcm_data.PatientID})
  patient_data.update({"Patient Name" :dcm_data.PatientName})
  patient_data.update({"Patient Sex" :dcm_data.PatientSex})

  if dcm_data.PatientBirthDate == "":
    patient_birth_date = ""
    patient_age_now = ""
  else:
    have_patient_dob = True
    birth_date = datetime.utcfromtimestamp(int(dcm_data.PatientBirthDate))
    if birth_date <= datetime.now():
      age_years, age_months, age_days = calculate_age(birth_date, datetime.now())
      patient_age_now = str(age_years) + " years, " + str(age_months) + " months, " + str(age_days) + " days"
      patient_birth_date = birth_date.strftime('%Y-%m-%d')

  if dcm_data.AcquisitionDate == "":
      acquisition_date = ""
      acquisition_age = ""
  else:
    have_acquisition_date = True
    acquisition_date = datetime.utcfromtimestamp(int(dcm_data.AcquisitionDate))
    patient_acquisition_date = acquisition_date.strftime('%Y-%m-%d')

  if (have_patient_dob == True and have_acquisition_date == True):

    if birth_date <= acquisition_date:
      age_years, age_months, age_days = calculate_age(birth_date, acquisition_date)
      acquisition_age = str(age_years) + " years, " + str(age_months) + " months, " + str(age_days) + " days"

  patient_data.update({"Patient Birth Date" :( patient_birth_date)})
  patient_data.update({"Patient Current Age" :( patient_age_now)})
  patient_data.update({"Series Number" :dcm_data.SeriesNumber})
  patient_data.update({"Acquisition Number" :dcm_data.AcquisitionNumber})
  patient_data.update({"Acquisition Date" : ( patient_acquisition_date)})
  patient_data.update({"Instance Number" :dcm_data.InstanceNumber})
  patient_data.update({"View Position" :dcm_data.ViewPosition})
  patient_data.update({"Patient Orientation" :dcm_data.PatientOrientation})
  patient_data.update({"Patient Age at Time of Acquisition" :( acquisition_age)})

  return patient_data


path = "/content/drive/MyDrive/Shared Capstone/Colab/mimic_images/demo-files"
file_name = "demo2.dcm"

# Convert dcm image to jpg:

ds = pydicom.dcmread(path + "/" + file_name)
dcm_data = pydicom.dcmread(path + "/" + file_name)
new_image = ds.pixel_array.astype(float)
scaled_image = (np.maximum(new_image, 0) / new_image.max()) * 255.0
scaled_image = np.uint8(scaled_image)
final_image = Image.fromarray(scaled_image)
final_image.save('image.jpg')

# # #read the image
image = cv.imread("image.jpg")
#convert the image to RGB (images are read in BGR in OpenCV)
RGB = cv.cvtColor(image, cv.COLOR_BGR2RGB)
new_image = Image.fromarray(RGB)
plt.imshow(new_image);
new_image = new_image.resize((1024, 1024))
new_image.save('new_image.jpg')

img = skimage.io.imread('new_image.jpg')
img = xrv.datasets.normalize(img, 255) # convert 8-bit image to [-1024, 1024] range
img = img.mean(2)[None, ...] # Make single color channel

transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])
img = transform(img)
img = torch.from_numpy(img)

# Load model and process image
model = xrv.models.DenseNet(weights="densenet121-res224-mimic_ch") # MIMIC-CXR (MIT)
#model = xrv.models.DenseNet(weights="densenet121-res224-mimic_nb") # MIMIC-CXR (MIT)
outputs = model(img[None,...]) # or outputs = model.features(img[None,...])
probability_dictionary = dict(zip(model.pathologies,outputs[0].detach().numpy()))

dicom_header_information_dictionary = dcm_patient_data(path + "/" + file_name)

print("\n")
print(tabulate((dicom_header_information_dictionary.items())))

line_content = []

with open("/content/drive/MyDrive/Shared Capstone/Colab/mimic_images/mimic-cxr-2.0.0-chexpert.csv") as myFile:
    for num, line in enumerate(myFile, 0):
        if dicom_header_information_dictionary["Study ID"] in line:
            line_content = line.split(",")
            print("the line content is: ", line_content)

Atelectasis_index = 2
Cardiomegaly_index = 3
Consolidation_index = 4
Edema_index = 5
Pleural_Effusion_index = 11

actual_dictionary = {
    'Atelectasis': 0,
    'Cardiomegaly': 0,
    'Consolidation': 0,
    'Edema': 0,
    'Effusion': 0
}

if (line_content[Atelectasis_index] == "1.0"): actual_dictionary['Atelectasis'] = 1
if (line_content[Cardiomegaly_index] == "1.0"): actual_dictionary['Cardiomegaly'] = 1
if (line_content[Consolidation_index] == "1.0"): actual_dictionary['Consolidation'] = 1
if (line_content[Edema_index] == "1.0"): actual_dictionary['Edema'] = 1
if (line_content[Pleural_Effusion_index] == "1.0"): actual_dictionary['Effusion'] = 1

array1 = ['Atelectasis', probability_dictionary['Atelectasis'], actual_dictionary['Atelectasis']]
array2 = ['Cardiomegaly', probability_dictionary['Cardiomegaly'], actual_dictionary['Cardiomegaly']]
array3 = ['Consolidation', probability_dictionary['Consolidation'], actual_dictionary['Consolidation']]
array4 = ['Edema', probability_dictionary['Edema'], actual_dictionary['Edema']]
array5 = ['Effusion', probability_dictionary['Effusion'], actual_dictionary['Effusion']]

table = [['Disease', 'Probability', 'Actual'], array1, array2, array3, array4, array5]

print("\n")
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))